## Configuration
_Initial steps to get the notebook ready to play nice with our repository. Do not delete this section._

Code formatting with [black](https://pypi.org/project/nb-black/).

In [1]:
%load_ext lab_black

Add our `utils` directory to the system's `$PATH` so we can import Python files from sibling directories.

In [2]:
import os
import pathlib

In [3]:
this_dir = pathlib.Path(os.path.abspath(""))

In [4]:
data_dir = this_dir / "data"

In [5]:
import pytz
import glob
import requests
import pandas as pd
from bs4 import BeautifulSoup
import unicodedata
import regex as re
from datetime import datetime
from dateutil.parser import parse

## Download

Retrieve the page

In [6]:
url = "https://www.amadorgov.org/services/covid-19/-fsiteid-1"

In [7]:
page = requests.get(url)

In [8]:
soup = BeautifulSoup(page.content, "html.parser")

Get content well

In [9]:
content = soup.find("div", id="widget_4_3778_3488")

Get timestamp

In [10]:
# imgs = [
#     c
#     for c in content.find_all("img")
#     if "Screenshot" and "Top" in c.attrs.get("alt", "")
# ]

In [11]:
imgs = content.find_all("img")

In [12]:
imgs

[<img alt="COVID-19 image 4.21.20" height="373" src="https://www.amadorgov.org/home/showpublishedimage/4397/637230569666270000" style="width: 469px; height: 180px;" width="810"/>,
 <img alt="Screenshot May 24 2022_Top" src="https://www.amadorgov.org/home/showpublishedimage/5061" style="margin: 15px; width: 640px; height: 784px;"/>,
 <img alt="Screenshot May 24 2022_Bottom" src="https://www.amadorgov.org/home/showpublishedimage/5063" style="margin: 15px; width: 840px; height: 766px;"/>,
 <img alt="Road_to Recovery Resources label" src="https://www.amadorgov.org/home/showpublishedimage/4431/637246532725070000" style="width: 380px; height: 254px;"/>,
 <img alt="Press Releases" src="https://www.amadorgov.org/home/showpublishedimage/3409/637229943791030000" style="width: 330px; height: 185px;"/>,
 <img alt="3D Coronavirus_Emergency Declarations" src="https://www.amadorgov.org/home/showpublishedimage/3413/637229944542970000" style="width: 330px; height: 185px;"/>,
 <img alt="3D Coronavirus i

In [13]:
url = f"https://www.amadorgov.org{imgs[1]['src']}"

In [14]:
tz = pytz.timezone("America/Los_Angeles")

In [15]:
today = datetime.now(tz).date()

In [16]:
with open(data_dir / "amador" / f"{today}.png", "wb") as handler:
    handler.write(requests.get(url).content)

ConnectionError: HTTPSConnectionPool(host='www.amadorgov.orghttps', port=443): Max retries exceeded with url: //www.amadorgov.org/home/showpublishedimage/5061 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fe7d4cc7c70>: Failed to establish a new connection: [Errno -2] Name or service not known'))